This jupyter notebook analyses the differences between the sexes by age in Ireland.
- Weighted mean age (by sex)
- The difference between the sexes by age

In [ ]:
# Importing the pandas library
import pandas as pd

In [ ]:
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)

In [ ]:
df.columns


In [ ]:
headers = df.columns.tolist()
headers

In [4]:
drop_columns = [
    'STATISTIC',
    'Statistic Label',
    'TLIST(A1)',
    'CensusYear',
    'C02199V02655',
    'C02076V03371',
    'C03789V04537',
    'Administrative Counties',
    'UNIT'
]


df.drop(columns=drop_columns, inplace=True)
df.columns


Index(['Sex', 'Single Year of Age', 'VALUE'], dtype='object')

In [ ]:
print(df.head(3))

Choosing just males and females. Ignoring Both sexes

In [ ]:
df = df[df["Sex"] != "Both sexes"]
df["Sex"].unique()

Calculation for male

In [ ]:
#df_male = df[df["Sex"] == "Male"]
#total_age = (df_male["Single Year of Age"] * df_male["VALUE"]).sum()
#total_population = df_male["VALUE"].sum()
#weighted_mean_male = total_age / total_population
